In [11]:
import os
import jax
from tqdm import trange
import jax.numpy as jnp
import numpy as np

from jax import jit, pmap, vmap, value_and_grad
from jax.tree_util import tree_map, tree_multimap
from jax.lax import psum, pmean
import matplotlib.pyplot as plt
import haiku as hk
import optax

import sklearn.datasets as dt

import wandb

wandb.init(project='jax_demo', entity='phamid')
# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.005

#jax.devices(),len(jax.devices())

wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [12]:
seed = 1337
features_num = 35
samples_num = 1000
rand = np.random.RandomState(seed) 


x,y_true = dt.make_friedman1(n_samples=samples_num,n_features=features_num,noise=0.2,random_state=rand)

In [13]:
x.shape, y_true.shape

((1000, 35), (1000,))

In [14]:
#y_true = m_true * x + b_true
#y_true = np.dot(m_true,x) + b_true

def model_fn(x):
  y = hk.Linear(samples_num)  # change number of outputs 

  return y(x)

model = hk.without_apply_rng(hk.transform(model_fn))

init_key = jax.random.PRNGKey(1337)
representation_input = jnp.ones((samples_num, features_num)) # model param variable x1  
params = model.init(init_key, representation_input)

opt = optax.adam(0.01)
opt_state = opt.init(params)

type(opt_state), type(params), y_true.shape, params['linear']['w'].shape,params['linear']['b'].shape

#'linear/w' with shape (1, 5) does not match shape=[5, 5] dtype=dtype('float64')


(list, haiku._src.data_structures.FlatMapping, (1000,), (35, 1000), (1000,))

In [20]:
#print(f"initial params w={params['linear']['w']}) b={params['linear']['b']}")

def mean_sqrd_loss (params,x, y_true):
    y_pred = model.apply(params, x)
    loss = (y_pred - y_true) **2
    return jnp.mean(loss)

def update (params, opt_state):
    loss_value, grads = value_and_grad(mean_sqrd_loss)(params, x, y_true)
    updates, opt_state = opt.update(grads, opt_state)
    params = optax.apply_updates(params, updates)

    return params, opt_state, loss_value

update = jit(update)

#loop thru dataset
epoch = trange(50)
for _ in epoch:
    for _ in range(100):
        params, opt_state, loss_value = update(params, opt_state)
    epoch.set_postfix({'Loss' : loss_value})
    
    #wandb.log({"loss": loss_value.copy()})

    
    

#print(f"final params w={params['linear']['w']}) b={params['linear']['b']}")

    


100%|██████████| 50/50 [00:00<00:00, 70.74it/s, Loss=9.719577e-06]
